This project is inspired by the idea of Lucas: Clustering and Visualisation using Folium Maps (https://www.kaggle.com/lucaspcarlini/clustering-and-visualisation-using-folium-maps)

library

In [5]:
import warnings 
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

import seaborn as sns
from matplotlib import pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import folium

from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
from hmmlearn import hmm

# ** load dataset**

this is a countrywide weather events dataset with more than 5 million events, which covers 49 states of the United States. Examples of weather events are rain, snow, storm, and freezing condition. The data is collected from January 2016 to December 2019, using historical weather reports that exist for airport-based weather stations across the country.

Weather event is a spatiotemporal entity, where such an entity is associated with location and time. Following is the description of available weather event types in this dataset: Cold: The case of having extremely low temperature, with temperature below -23.7 degrees of Celsius.

Fog: The case where there is low visibility condition as a result of fog or haze.

Hail: The case of having solid precipitation including ice pellets and hail.

Rain: The case of having rain, ranging from light to heavy.

Snow: The case of having snow, ranging from light to heavy.

Storm: The extremely windy condition, where the wind speed is at least 60 km/h.

Other Precipitation: Any other type of precipitation which cannot be assigned to previously described event types.

In [6]:
df = pd.read_csv('../input/us-weather-events/US_WeatherEvents_2016-2019.csv')
df.tail(3)

FileNotFoundError: [Errno 2] File b'../input/us-weather-events/US_WeatherEvents_2016-2019.csv' does not exist: b'../input/us-weather-events/US_WeatherEvents_2016-2019.csv'

# data cleaning

In this project, the duration of each weather event was the key feature used to cluster regions. It was calculated by using event end time minus start time, any single event that lasted more than 30 days was initially eliminated, then any events outside three standard deviation away from mean were also removed.

In [3]:
datetimeFormat = '%Y-%m-%d %H:%M:%S'
df['End']=pd.to_datetime(df['EndTime(UTC)'], format=datetimeFormat)
df['Start']=pd.to_datetime(df['StartTime(UTC)'], format=datetimeFormat)
df['Duration']=df['End']-df['Start']
df['Duration'] = df['Duration'].dt.total_seconds()
df['Duration'] = df['Duration']/(60*60) #in hours
df = df[(df['Duration']< 30*24) & (df['Duration'] != 0)] #remove obvious wrong data
df.tail(3)

,EventId,Type,Severity,StartTime(UTC),EndTime(UTC),TimeZone,AirportCode,LocationLat,LocationLng,City,County,State,ZipCode,End,Start,Duration
5059830,W-5060509,Fog,Moderate,2019-12-28 12:53:00,2019-12-28 13:35:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2019-12-28 13:35:00,2019-12-28 12:53:00,0.700000
5059831,W-5060510,Snow,Light,2019-12-28 13:35:00,2019-12-28 17:04:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2019-12-28 17:04:00,2019-12-28 13:35:00,3.483333
5059832,W-5060511,Snow,Light,2019-12-28 17:15:00,2019-12-28 17:53:00,US/Mountain,KBVR,42.5833,-108.2833,Lander,Fremont,WY,82520.0,2019-12-28 17:53:00,2019-12-28 17:15:00,0.633333


# Check events distribution

In [4]:
print("Overall Duration Summary")
print("--Count", df['Duration'].size)
print("--%miss.", sum(df['Duration'].isnull()))
print("--card.",df['Duration'].unique().size)
print("--min",df['Duration'].min())
print("--lowerBoundary.",df['Duration'].median()-1.5*((df['Duration'].quantile(0.75))-df['Duration'].quantile(0.25)))
print("--1stQrt",df['Duration'].quantile(0.25))
print("--mean",df['Duration'].mean())
print("--median",df['Duration'].median())
print("--3rdQrt",df['Duration'].quantile(0.75))
print("--upperBoundary.",df['Duration'].median()+1.5*((df['Duration'].quantile(0.75))-df['Duration'].quantile(0.25)))
print("--95%Boundary.",df['Duration'].mean()+1.96*df['Duration'].std())
print("--max",df['Duration'].max())
print("--Std.Dev",df['Duration'].std())

Overall Duration Summary
--Count 5059623
--%miss. 0
--card. 3611
--min 0.01666666666666667
--lowerBoundary. -0.7583333333333336
--1stQrt 0.3333333333333333
--mean 1.3141716849822662
--median 0.6666666666666666
--3rdQrt 1.2833333333333334
--upperBoundary. 2.091666666666667
--95%Boundary. 10.119511374248747
--max 718.6666666666666
--Std.Dev 4.4925202496257555


In [5]:
df = df[(df['Duration']< 10)]
df['Duration'].size

5004602

**data normalization**
For better comparison of each weather events, all the duration time of them were normalized to the range 0 to 100, zero means that event never happened in the year and 100 means that event happened all the time during the year.

In [6]:
df2 = df.groupby(['AirportCode','City','State', 
                  'LocationLat', 'LocationLng','Type']).agg({'Duration':['sum']}).reset_index()
df2.columns=pd.MultiIndex.from_tuples((("AirportCode", " "),("City", " "),
                                       ("State", " "), ("LocationLat", " "),
                                       ("LocationLng", " "), ("Type", " "), ("Duration", " ")))
df2.columns = df2.columns.get_level_values(0)
df2['Duration'] = df2['Duration']/(24*4*3.65) #yearly percentage  
df2 = df2.sort_values(by='Duration')
df2.tail(3)

,AirportCode,City,State,LocationLat,LocationLng,Type,Duration
272,K3TH,Thompson Falls-West End,MT,47.6000,-115.3667,Snow,14.227454
1087,KAST,Warrenton,OR,46.1569,-123.8825,Rain,14.638508
11026,KUIL,Forks,WA,47.9375,-124.5550,Rain,16.484494


In [7]:
df_flat = df2.pivot(index='AirportCode', columns='Type', values=['Duration']).reset_index().fillna(0)
df_flat.columns=pd.MultiIndex.from_tuples(((' ', 'AirportCode'),(' ', 'Cold'),(' ', 'Fog'),
            (' ',  'Hail'),(' ', 'Precipitation'),(' ', 'Rain'),(' ', 'Snow'),(' ', 'Storm')))
df_flat.columns = df_flat.columns.get_level_values(1)
#df_flat().tail(3)
uniqueKey = df2[['AirportCode', 'City', 
                 'State', 'LocationLat', 'LocationLng']].sort_values(by='AirportCode').drop_duplicates()
weather = pd.merge(df_flat, uniqueKey, how='inner', on='AirportCode')
weather.tail(3)

,AirportCode,Cold,Fog,Hail,Precipitation,Rain,Snow,Storm,City,State,LocationLat,LocationLng
2064,KYNG,0.019692,2.263223,0.016553,0.065401,9.504138,4.509751,0.000666,Vienna,OH,41.2544,-80.6739
2065,KZPH,0.242580,1.933029,0.000000,0.038052,4.705099,0.000000,0.001903,Zephyrhills,FL,28.2281,-82.1559
2066,KZZV,0.005708,1.831573,0.000000,0.055889,6.204005,1.820348,0.000000,Zanesville,OH,39.9444,-81.8921


# **Exploratory data analysis**

# **National wide weather events duration**

The national wide weather event duration information was shown in figure 1. Rain lasted around 4.5% period of a year, which is the most durable weather event. Fog and snow lasted around 1.5% time of a year. The duration of cold, precipitation, storm and hail were all below 0.5%.

In [8]:
fig_sum = px.histogram(df2, x='Type', y= 'Duration',  histfunc = 'avg',
                      title = 'fig 1. National wide weather events duration')
fig_sum.update_xaxes(categoryorder='mean descending')
fig_sum.update_yaxes(title_text='mean of duration% per year')
fig_sum.update_layout(height=750, width=1000)
fig_sum.show()

# **State wide weather events distribution**

The statewide weather event duration information was shown in figure 2.

The top three states with long rainy days are OR, DE, MA; The top three states with long foggy days are NH, WA, CA; The top three states with long snow days are MI, VT, MT; The top three states with long cold days are ND, MN, NJ; The top three states with long precipitation days are NY, FL, LA; The top three states with long storm days are NH, CO, WY; The top three states with long hail days are CT, NY, DE.

The top three states with short rainy days are NV, UT, AZ; The top three states with short foggy days are AZ, NV, NM; The top three states with short snow days are FL, LA, TX; The top three states with short cold days are MO, WA, CT; The top three states with short precipitation days are UT, NV, CO; The top three states with short storm days are AL, GA, KY; The top three states with short hail days are FL, AZ, GA.

All of these information matched our common sense.

In [9]:
fig_state=make_subplots(rows=7, cols=1, shared_yaxes=False, vertical_spacing=0.05)

fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Rain'], name='Rain', histfunc ='avg'),1,1)
fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Fog'], name='Fog', histfunc ='avg'),2,1)
fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Snow'], name='Snow', histfunc ='avg'),3,1)
fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Cold'], name='Cold', histfunc ='avg'),4,1)
fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Precipitation'], name='Precipitation', histfunc ='avg'),5,1)
fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Storm'], name='Storm', histfunc ='avg'),6,1)
fig_state.add_trace(go.Histogram(x=weather['State'], y=weather['Hail'], name='Hail', histfunc ='avg'),7,1)

fig_state['layout']['xaxis7'].update(title="State")
fig_state['layout']['yaxis4'].update(title="duration% per year")
fig_state.update_xaxes(categoryorder='mean descending')
fig_state.update_layout( title_text="fig 2. State wide weather events distribution")
fig_state.show()

# **City wide weather events distribution**

# **cities involved**

All the cities involved in this dataset are marked as blue dot in figure 3. Large amount of cities distribued in west coast and east part of the United States, few of them located in the middle.

In [10]:
fig_city = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      hover_name=weather['City'] + ', ' + weather['State'],
                      scope="usa",
                      title ='fig 3. Cities involved in this dataset')
#fig_city.update_layout(height=750, width=1000)
fig_city.show()

# **Rain**

The city-wide rain distribution is shown in figure 4. United States cities with long rainy durations are distributed at the west coast and east areas (dark blue markers), cities in the middle part of the region has lower chance of raining (light blue area).

In [11]:
fig_rain = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Rain",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense', 
                      range_color = [0,16],
                      scope="usa",
                      title ='fig 4. City wide rainy days percentage each year from 2016 to 2019')
#fig_rain.update_layout(height=750, width=1000)
fig_rain.show()

# **Fog**

The city-wide fog distribution is shown in figure 5. Like the distribution of rain, cities with long fog days are distributed at the west coast and east areas (dark blue markers), cities in the middle part of the United States has lower chance of fog (light blue area).

In [12]:
fig_fog = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Fog",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense', 
                      #range_color = [0,16],
                      scope="usa",
                      title ='fig 5. City wide foggy days percentage each year from 2016 to 2019')
#fig_fog.update_layout(height=750, width=1000)
fig_fog.show()

# **Snow**

The city-wide snow distribution is shown in figure 6. Cities in the north and middle part of the United States have higher chance of snow (dark blue markers), cities in the south east area have lower chance of snow (light blue area).

In [13]:
fig_snow = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Snow",
                      #size="Snow",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense', 
                      #range_color = [0,16],
                      scope="usa",
                      title ='fig 6. City wide snow days percentage each year from 2016 to 2019')
#fig_snow.update_layout(height=750, width=1000)
fig_snow.show()

# **Cold**

Cold is defined as the case of having extremely low temperature, with temperature below -23.7 degrees of Celsius. The city-wide cold distribution is shown in figure 7. Cities in the north part of the United States have higher chance of cold, it is consistent with our common sense. However, there are large numbers of cities with high chance of cold distributed in the south east area, which is counter intuitive, thus the weather event cold may not be a good feature to cluster regions, and it was removed for the followed analysis.

In [14]:
fig_cold = px.scatter_geo(weather, lat='LocationLat', lon='LocationLng',
                      color="Cold",
                      #size="Snow",
                      hover_name=weather['City'] + ', ' + weather['State'],
                      color_continuous_scale='dense', 
                      #range_color = [0,16],
                      scope="usa",
                      title ='fig 7. City wide cold days percentage each year from 2016 to 2019')
#fig_cold.update_layout(height=750, width=1000)
fig_cold.show()

# **K means clustering**
 **algorithm**

K-means algorithm is one of the most employed clustering algorithms. K-means aims to partition N observations into K clusters in which each observation belongs to the cluster with the nearest mean. The algorithm proceeds as follows:

Pick K random points as cluster center positions.

Assign each point to the nearest center.

Recompute each cluster mean as the mean of the vectors assigned to that cluster.

If centers moved, go to step 2.

**Distance**

The algorithm requires a distance measure to be deﬁned in the data space, and the Euclidean distance is often used. For example, the Euclidean distance between u = (u 1 , u 2 ) and v = (v 1 , v 2 ) is calculated by the following expression:

r(u, v) = √（(u1-v1)2+(u2-v2)2

**Pick K random points as cluster center position**

The elbow method is used to determine the optimal number of clusters. It plots the value of the cost function produced by different values of k. When k increases, the average distortion will increase, each cluster will have fewer constituent instances, and the instances will be closer to their respective centroids. However, the improvements in average distortion will decline as k increases. The value of k at which improvement in distortion declines the most is called the elbow, at which we should stop dividing the data into further clusters [1].

From figure 8, the elbow happened when k = 3 or 4. Thus I will pick 4 as the number of clusters in this dataset.

In [15]:
X = df_flat.drop(['AirportCode','Cold', 'Hail'], axis=1)
X.tail(3)

,Fog,Precipitation,Rain,Snow,Storm
2064,2.263223,0.065401,9.504138,4.509751,0.000666
2065,1.933029,0.038052,4.705099,0.000000,0.001903
2066,1.831573,0.055889,6.204005,1.820348,0.000000


In [16]:
from sklearn.cluster import KMeans
distortions = []

K = range(1,20)
for k in K:
    kmean = KMeans(n_clusters=k, random_state=0, n_init = 50, max_iter = 500)
    kmean.fit(X)
    distortions.append(kmean.inertia_)

fig_kmean = px.scatter(x=K, y=distortions, trendline=distortions, title='fig 8. The Elbow Method')
fig_kmean.update_xaxes(title_text='k')
fig_kmean.update_yaxes(title_text='distortion')
#fig_kmean.update_layout(height=650, width=1000)
fig_kmean.show()

# **K means clustering results**

**city-wide cluster distribution**

The city-wide results of K-means clustering is shown in figure 9. It seems k=4 is a good number of clusters. And cluster 0 (blue), 2 (green) and 3 (purple) have clear boundary with each other. Cities in cluster 1 (red) widely spread in the United States.

In [17]:
kmeans = KMeans(n_clusters=4, random_state=0).fit(X)

df_flat['Cluster'] = (kmeans.labels_).astype(str)
df_cluster = pd.merge(df_flat[['AirportCode','Cluster']], weather.drop(['Cold','Hail'], axis=1), 
                      how='inner', on='AirportCode')
df_cluster.tail(3)

,AirportCode,Cluster,Fog,Precipitation,Rain,Snow,Storm,City,State,LocationLat,LocationLng
2064,KYNG,3,2.263223,0.065401,9.504138,4.509751,0.000666,Vienna,OH,41.2544,-80.6739
2065,KZPH,0,1.933029,0.038052,4.705099,0.000000,0.001903,Zephyrhills,FL,28.2281,-82.1559
2066,KZZV,3,1.831573,0.055889,6.204005,1.820348,0.000000,Zanesville,OH,39.9444,-81.8921


In [18]:
fig_cluster = px.scatter_geo(df_cluster, lat='LocationLat', lon='LocationLng',
                      hover_name=weather['City'] + ', ' + weather['State'],
                      scope="usa",
                      color_discrete_sequence =['#AB63FA', '#EF553B', '#00CC96','#636EFA'],
                      color = 'Cluster',
                      title ='fig 9. City wide weather cluster distribution')
#fig_cluster.update_layout(height=750, width=1000)
fig_cluster.show()

**state-wide cluster distribution**

To better visulize each cluster's distribution in each state, clusters are grouped as state and location was calculated as the average of each cities latidude and logitude in that cluster. The state wide cluster distribution is shown in figure 10. Almost all states have mixtured types of clusters.

In [19]:
df_cluster2 = df_cluster.groupby(['State','Cluster']).agg({'Cluster':['count']}).reset_index()
df_cluster2.columns=pd.MultiIndex.from_tuples((("State", " "),("Cluster", " "),("Count", " ")))
df_cluster2.columns = df_cluster2.columns.get_level_values(0)
#df_cluster2.tail(3) #state with each cluster counts

df_loc = df_cluster[['State','Cluster','LocationLat', 'LocationLng']]
df_loc1 = df_loc.groupby(['State','Cluster']).agg({'LocationLat':'mean'}).reset_index()
df_loc2 = df_loc.groupby(['State','Cluster']).agg({'LocationLng':'mean'}).reset_index()
df_loc3 = pd.merge(df_loc1,df_loc2, how='inner', on=['State','Cluster'])
#df_loc3.tail(3) #state with cluster and location

df_clusterS = pd.merge(df_loc3,df_cluster2, how='inner', on=['State','Cluster'])
df_clusterS.tail(3) #state with each cluster count location

,State,Cluster,LocationLat,LocationLng,Count
139,WY,1,42.639810,-107.355340,30
140,WY,2,42.824439,-107.523865,23
141,WY,3,41.725300,-106.459400,1


In [20]:
fig_clusterS = px.scatter_geo(df_clusterS, lat='LocationLat', lon='LocationLng', 
                     color='Cluster',
                     size='Count',
                     color_discrete_sequence=['#636EFA', '#AB63FA', '#EF553B','#00CC96'],
                     hover_name='State',
                     scope="usa",
                     title = 'fig 10. State wide weather cluster distribution')
#fig_clusterS.update_layout(height=750, width=1000)
fig_clusterS.show()

**property of each cluster**

The property of each cluster is summarized in figure 11. Compare to other clusters, cluster 3 (purple) has highest chance of rain, cluster 2 (green) has highest chance of snow, cluster 1 (red) has lowest chance of rain while highest chance of storm; cluster 0 (blue) is similar to cluster 3 while its’ chance of rain is slightly lower than cluster 3.

In [21]:
prop = df_cluster[['Cluster', 'Fog',
                   'Precipitation','Rain', 'Snow', 'Storm']].groupby(['Cluster']).mean().reset_index()
prop2 = prop.transpose().reset_index()
prop2 = prop2[(prop2['index'] !='Cluster')].sort_values(by=0)
prop2

,index,0,1,2,3
5,Storm,0.0282847,0.0819425,0.0542485,0.00993557
2,Precipitation,0.06596,0.0522603,0.0275178,0.0597302
4,Snow,0.392509,0.710154,3.7227,0.808844
1,Fog,1.84685,0.900525,1.8588,1.81258
3,Rain,4.31809,1.62774,4.50697,6.90378


In [22]:
#from plotly.subplots import make_subplots
fig_prop=make_subplots(rows=1, cols=4, shared_yaxes=True,horizontal_spacing=0)

fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[0], name='Cluster 0'), row=1, col=1)
fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[1], name='Cluster 1'), row=1, col=2)
fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[2], name='Cluster 2'), row=1, col=3)
fig_prop.add_trace(go.Bar(x=prop2['index'], y=prop2[3], name='Cluster 3'), row=1, col=4)

fig_prop.update_yaxes(title_text="duration%/year", row=1, col=1)
fig_prop.update_layout(title_text="fig 11. Weather distribution in each cluster")
#fig_prop.update_layout(height=550, width=1000)
fig_prop.show()

**representive cities in each cluster**

Four cities were selected in each cluster, Kansas City, MO is the representive city of clsuter 0; Dever, CO is the representive city of cluster 1; Detroit, MI is the representive city of cluster 2; Seattle, MA is the representive city of cluster 3. All the cities weather information are shown in figure 10, their weather properties are similar to their corresponding cluster as shown in figure 12.

In [23]:
df3 = weather[(weather['City']=='Seattle')| (weather['City']=='Detroit')|(weather['City']== 'Kansas City')|(weather['City']== 'Denver')]
#df3
df4 = df3[['Storm', 'Precipitation','Snow', 'Fog','Rain', 'City']].groupby(['City']).mean().reset_index()
df4 = df4.transpose().reset_index()
df4.columns = df4.iloc[0]
df4 = df4[(df4['City'] !='City')]
df4

,City,Denver,Detroit,Kansas City,Seattle
1,Storm,0.0928938,0.010821,0.00239409,0.00126046
2,Precipitation,0.0189783,0.0506802,0.0847127,0.014531
3,Snow,2.76327,2.94295,0.724426,0.406749
4,Fog,1.54814,2.40287,0.729896,1.28089
5,Rain,2.28877,6.18612,3.45446,9.89852


In [24]:
fig_city=make_subplots(rows=1, cols=4, shared_yaxes=True,horizontal_spacing=0)

fig_city.add_trace(go.Bar(x=df4['City'], y=df4['Kansas City'], name='Cluster0'), row=1, col=1)
fig_city.add_trace(go.Bar(x=df4['City'], y=df4['Denver'], name='Cluster1'), row=1, col=2)
fig_city.add_trace(go.Bar(x=df4['City'], y=df4['Detroit'], name='Cluster2'), row=1, col=3)
fig_city.add_trace(go.Bar(x=df4['City'], y=df4['Seattle'], name='Cluster3'), row=1, col=4)

fig_city['layout']['xaxis1'].update(title="Kansas City, MO")
fig_city['layout']['xaxis2'].update(title="Denver, CO")
fig_city['layout']['xaxis3'].update(title="Detroit, MI")
fig_city['layout']['xaxis4'].update(title="Seattle, WA")
fig_city.update_yaxes(title_text="duration%/year", row=1, col=1)
fig_city.update_layout(title_text="fig 12. Representative cities in each cluster")
#fig_city.update_layout(height=550, width=1000)
fig_city.show()

# **Principle component analysis**
A way for obtaining a better understanding of the data is to visualize it; however, it is not straightforward to visualize high-dimensional data. This was a problem, for example, we could only choose pairs of weather events and plot them.

Principal Component Analysis (PCA) is a technique that is widely used for applications such as dimensionality reduction, visualization and lossy data compression. This project only focused on the visualization aspect. PCA can be deﬁned as the orthogonal linear transformation of the data to a lower dimensional linear space, known as the principal subspace, such that the greatest variance by any projection of the data comes to lie on the ﬁrst coordinate (called the ﬁrst principal component), the second greatest variance on the second coordinate, and so on. Intuitively, PCA ﬁnds a meaningful coordinate basis to express the dataset [2].

In order to visualize how are the clusters are related in the original high dimension space, PCA was applied to the original dataset. The result of PCA is shown in figure 11, the cumulative variance explained for the first three component is able to cover more than 95% of the original information, therefore, reducing the dimensionality of the dataset (figure 13 left). After mapping the cluster labels to the PCA dataset and visualize it in three dimensions (figure 13 right), it showed that four cluster is a reasonable to identify similar samples within the dataset.

In [25]:
pca = PCA().fit(X)
pcaX = pca.transform(X)
c0 = []
c1 = []
c2 = []
c3 = []

for i in range(len(pcaX)):
    if kmeans.labels_[i] == 0:
        c0.append(pcaX[i])
    if kmeans.labels_[i] == 1:
        c1.append(pcaX[i])
    if kmeans.labels_[i] == 2:
        c2.append(pcaX[i])
    if kmeans.labels_[i] == 3:
        c3.append(pcaX[i])
c0 = np.array(c0)
c1 = np.array(c1)
c2 = np.array(c2)
c3 = np.array(c3)

fig_pca = make_subplots(rows=1, cols=2, column_widths=[0.3, 0.7], specs=[[{'type':'domain'}, {'type': 'mesh3d'}]])

fig_pca.add_trace(go.Pie(values = pca.explained_variance_ratio_), row=1, col=1)

fig_pca.add_trace(go.Scatter3d(x=c0[:,0], y=c0[:,1], z=c0[:,2],
                               mode='markers', name='Cluster0'),  row=1, col=2)
fig_pca.add_trace(go.Scatter3d(x=c1[:,0], y=c1[:,1], z=c1[:,2], 
                               mode='markers', name='Cluster1'),  row=1, col=2)
fig_pca.add_trace(go.Scatter3d(x=c2[:,0], y=c2[:,1], z=c2[:,2], 
                               mode='markers', name='Cluster2'),  row=1, col=2)
fig_pca.add_trace(go.Scatter3d(x=c3[:,0], y=c3[:,1], z=c3[:,2], 
                               mode='markers', name='Cluster3'),  row=1, col=2)

fig_pca.update_layout(height=750, width=1000, title_text=
          "fig 13. PCA: explained variance%(left) and First 3 Component with mapped cluster(right)")

fig_pca.show()

# forecast Atlanta's weather events

In [26]:
atlanta = df[(df['City']== 'Atlanta')]
print(atlanta['LocationLat'].unique())
print(atlanta['LocationLng'].unique())

[33.6301 33.8784 33.7776]
[-84.4418 -84.298  -84.5246]


In [27]:
import folium
m = folium.Map(location=[33.755238, -84.388115])
folium.Marker(
    location=[33.6301, -84.4418],
).add_to(m)
folium.Marker(
    location=[33.8784, -84.298],
).add_to(m)
folium.Marker(
    location=[33.7776, -84.5246],
).add_to(m)
m

Atlanta has three airports, in this project will focus on the largest airport : Hartsfield-Jackson Atlanta International Airport.

The last observation of the event was in Dec 2019, different seasons have different weather properties, thus only events from month Nov, Dec and Jan were considered.

In [28]:
atlanta = atlanta[(atlanta['LocationLng']== -84.4418)]

atlanta_m = atlanta.loc[(atlanta['Start'].dt.month==12) | (atlanta['Start'].dt.month==11)| (atlanta['Start'].dt.month==1)]
atlanta_m = atlanta_m.loc[(atlanta_m['Type']!='Hail')&(atlanta_m['Type']!='Precipitation')]
print(atlanta_m['Type'].unique())
print(atlanta_m['Severity'].unique())
print(atlanta_m['Type'].count())

['Rain' 'Fog' 'Snow']
['Light' 'Moderate' 'Severe' 'Heavy']
680


last five events selected as testing dataset 

In [29]:
X2 = atlanta_m[['Type', 'Severity']]
X2_test = X2.tail(5)
X2_train = X2.head(675)
print(X2_train.tail())
print(X2_test)
print(X2['Severity'].describe())

         Type  Severity
3204817  Rain  Moderate
3204820   Fog    Severe
3204821   Fog    Severe
3204822  Rain     Light
3204823   Fog    Severe
         Type Severity
3204824  Rain    Light
3204825   Fog   Severe
3204826  Rain    Light
3204827  Rain    Light
3204828  Rain    Light
count       680
unique        4
top       Light
freq        414
Name: Severity, dtype: object


few events were in severe and heavy, since HMMlearn cannot handle observation states (rain, fog, etc.) bigger than hidden states (light, moderate, etc), thus combine severe and heavy together.

In [30]:
Type = {'Rain':0, 'Fog':1, 'Snow':2} 
X2.Type = [Type[item] for item in X2.Type] 

Severity = {'Light':0, 'Moderate':1, 'Severe':2, 'Heavy':2}
X2.Severity = [Severity[item] for item in X2.Severity] 
X2_test = X2.tail(5)
X2_train = X2.head(675)
print(X2_train.tail())
print(X2_test)

         Type  Severity
3204817     0         1
3204820     1         2
3204821     1         2
3204822     0         0
3204823     1         2
         Type  Severity
3204824     0         0
3204825     1         2
3204826     0         0
3204827     0         0
3204828     0         0


# Forecast weather events using Markov Chain

transition matrix

In [31]:
# A function that implements the Markov model to forecast the state. [3]
def transition_matrix(transitions):
    n = 1+ max(transitions) #number of states

    M = [[0]*n for _ in range(n)]

    for (i,j) in zip(transitions,transitions[1:]):
        M[i][j] += 1

    #now convert to probabilities:
    for row in M:
        s = sum(row)
        if s > 0:
            row[:] = [f/s for f in row]
    return M

#test:
t = list(X2_train.Type)
m = transition_matrix(t)
for row in m: print(' '.join('{0:.2f}'.format(x) for x in row))  

0.89 0.10 0.02
0.59 0.39 0.02
0.33 0.07 0.60


forecast weather using gained transition matrix

In [72]:
# The statespace
states = ["Rain","Fog","Snow"]

# Possible sequences of events
transitionName = [["RR","RF","RS"],["FR","FF","FS"],["SR","SF","SS"]]

# Probabilities matrix (transition matrix)
transitionMatrix = [[0.89,0.10,0.02],[0.59,0.39,0.02],[0.33,0.07,0.60]]

# A function that implements the Markov model to forecast the state. [4]
def activity_forecast(days):
    # Choose the starting state
    activityToday = "Fog"
    print("Start state: " + activityToday)
    # Shall store the sequence of states taken. So, this only has the starting state for now.
    activityList = [activityToday]
    i = 0
    # To calculate the probability of the activityList
    prob = 1
    while i != days:
        if activityToday == "Rain":
            change = np.random.choice(transitionName[0],replace=True,p=transitionMatrix[0])
            if change == "RR":
                prob = prob * 0.89
                activityList.append("Rain")
                pass
            elif change == "RF":
                prob = prob * 0.1
                activityToday = "Fog"
                activityList.append("Fog")
            else:
                prob = prob * 0.02
                activityToday = "Snow"
                activityList.append("Snow")
        elif activityToday == "Fog":
            change = np.random.choice(transitionName[1],replace=True,p=transitionMatrix[1])
            if change == "FR":
                prob = prob * 0.59
                activityList.append("Rain")
                pass
            elif change == "FF":
                prob = prob * 0.39
                activityToday = "Fog"
                activityList.append("Fog")
            else:
                prob = prob * 0.02
                activityToday = "Snow"
                activityList.append("Snow")
        elif activityToday == "Snow":
            change = np.random.choice(transitionName[2],replace=True,p=transitionMatrix[2])
            if change == "SR":
                prob = prob * 0.33
                activityList.append("Rain")
                pass
            elif change == "SF":
                prob = prob * 0.07
                activityToday = "Fog"
                activityList.append("Fog")
            else:
                prob = prob * 0.6
                activityToday = "Snow"
                activityList.append("Snow")
        i += 1  
    print("Possible states: " + str(activityList))
    print("Probability of the possible sequence of states: " + str(prob))
    
# Function that forecasts the possible state for the next 4 events
activity_forecast(4)
print('Acutal sequence of states: Rain, Fog, Rain, Rain, Rain')

Start state: Fog
Possible states: ['Fog', 'Fog', 'Rain', 'Rain', 'Rain']
Probability of the possible sequence of states: 0.08009780999999999
Acutal sequence of states: Rain, Fog, Rain, Rain, Rain


4/5 accuracy with 0.08 probability

In [80]:
activity_forecast(4)
print('Acutal sequence of states: Rain, Fog, Rain, Rain, Rain')

Start state: Fog
Possible states: ['Fog', 'Fog', 'Fog', 'Fog', 'Fog']
Probability of the possible sequence of states: 0.02313441
Acutal sequence of states: Rain, Fog, Rain, Rain, Rain


**1/5 accuracy with 0.02 probability

# predict weather severity using HMM

In [51]:
from hmmlearn import hmm
model = hmm.MultinomialHMM(n_components=3, n_iter=50, tol=0.001)
model.startprob_=np.array([0.82, 0.132, 0.044])
model.fit(np.array(X2_train))
#model.fit(X2_train['Type'], X2_train['Severity'])
print('startprob', np.round(model.startprob_,3))
print('transmat', np.round(model.transmat_,3))
print('emissionprob', np.round(model.emissionprob_,3))
print('score', model.score(X2))

startprob [0.983 0.017 0.   ]
transmat [[0.603 0.311 0.086]
 [0.567 0.341 0.092]
 [0.063 0.252 0.684]]
emissionprob [[0.854 0.143 0.003]
 [0.838 0.074 0.088]
 [0.24  0.447 0.313]]
score -1023.6267751366788


In [52]:
print ('Acutal sequence of states: 0, 2, 0, 0 ,0')
seen = np.array(X2_test[['Type']])
model.decode(seen, algorithm='viterbi')

Acutal sequence of states: 0, 2, 0, 0 ,0


(-4.614428217038643, array([0, 0, 0, 0, 0]))

4/5 accuracy with log(-4.6)=0.66 probability

# Reference

[1] https://www.oreilly.com/library/view/statistics-for machine/9781788295758/c71ea970-0f3c-4973-8d3a-b09a7a6553c1.xhtml

[2] https://setosa.io/ev/principal-component-analysis/

[3] https://stackoverflow.com/questions/46657221/generating-markov-transition-matrix-in-python

[4] https://www.datacamp.com/community/tutorials/markov-chains-python-tutorial